---
<div align="center">

# ML2 | Classificação de Sons Urbanos
</div>

---


### Contexto

``Classificação de Sons`` é considerada uma das tarefas mais importantes no campo de **deep learning**. Tem grande impacto em aplicações de **reconhecimento de voz** em assistentes virtuais (como Siri ou Alexa), **atendimento ao cliente**, bem como em **sistemas de recomendação de música e mídia**. Além disso, também influencia na área médica para detectar anomalias em batimentos cardíacos ou sons respiratórios. Além disso, é utilizada em **sistemas de segurança e vigilância** para ajudar a detectar e avaliar possíveis incidentes dentro de uma residência, como pedidos de socorro, disparos ou vidros quebrando. Portanto, nosso objetivo é desenvolver **algoritmos de deep learning** que nos permitam classificar corretamente alguns sons ambientais fornecidos pelo dataset ``UrbanSound8k``.

### Abordagem de Resolução

Utilizando o dataset ``UrbanSound8k`` como nossa fonte principal de dados, pretendemos projetar modelos de deep learning capazes de **identificar**, **prever** e **avaliar** diversos sons comuns em ambientes urbanos, apoiando aplicações em monitoramento de ruído, segurança pública e análise ambiental, permitindo **avaliação precisa** e resposta a eventos sonoros em ambientes urbanos.

### Plano

O projeto inclui várias fases-chave, incluindo:

1. ``Análise Exploratória de Dados``: Começamos analisar o dataset UrbanSound8k para entender melhor sua **estrutura e conteúdo** e a distribuição das classes sonoras.
2. ``Pré-processamento de Dados``: **Limpeza e preparação das amostras de áudio** para garantir consistência e qualidade.
3. ``Engenharia de Features``: Utilizar a biblioteca **Librosa**, extraímos **features relevantes** dos áudios, como as **coeficientes MFCC (Mel-frequency cepstral coefficients)**.
4. ``Definição de Arquitetura de Modelos``: Desenvolvemos arquiteturas de redes neurais adequadas para classificação de som, experimentando diferentes modelos de deep learning.
5. ``Treinamento e Avaliação de Desempenho``: Empregando validação cruzada com 10 folds no dataset pre-particionado para avaliar o desempenho dos modelos usando métricas como acurácia e matrizes de confusão.
6. ``Inferência Estatística``: Realizar uma avaliação estatística das diferenças de desempenho entre os modelos desenvolvidos.

<div align="center">
    
---
## Importar dataset
---

</div>

In [2]:
"""
# TensorFlow 2.15 compatível com Python 3.10
!pip install tensorflow==2.15.0 keras==2.15.0 tensorflow-hub==0.16.1

# Pacotes de suporte
!pip install numpy==1.24.3 pandas==1.5.3 scipy==1.10.1

# Soundata instala automaticamente a versão compatível de librosa
!pip install soundata
"""


'\n# TensorFlow 2.15 compatível com Python 3.10\n!pip install tensorflow==2.15.0 keras==2.15.0 tensorflow-hub==0.16.1\n\n# Pacotes de suporte\n!pip install numpy==1.24.3 pandas==1.5.3 scipy==1.10.1\n\n# Soundata instala automaticamente a versão compatível de librosa\n!pip install soundata\n'

In [3]:

import soundata

dataset = soundata.initialize("urbansound8k", data_home="E:/iacd/3ano/AC2")
print(dataset.data_home)

"""
dataset.download()  # download the dataset
dataset.validate()  # validate that all the expected files are there

example_clip = dataset.choice_clip()  # choose a random example clip
print(example_clip)  # see the available data

"""

E:/iacd/3ano/AC2


'\ndataset.download()  # download the dataset\ndataset.validate()  # validate that all the expected files are there\n\nexample_clip = dataset.choice_clip()  # choose a random example clip\nprint(example_clip)  # see the available data\n\n'

<div align="center">
    
---
## Importar bibliotecas
---

</div>

In [4]:
import pandas as pd
import librosa as libr

In [5]:
import tensorflow as tf
from tensorflow.keras.optimizers import (Adam)
from tensorflow.keras.callbacks import (EarlyStopping, ReduceLROnPlateau)
from tensorflow.keras import regularizers
import tensorflow_hub as hub
from scipy.stats import (friedmanchisquare)

In [6]:
import sys
print(sys.executable)

C:\Users\zni02\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe


<div align="center">
    
---
## Análise Exploratória de Dados
---

</div>

Nesta fase inicial do projeto, planejamos **revisar e analisar os dados** presentes no dataset UrbanSound8k. Nosso objetivo é obter melhores percepções não apenas sobre os dados disponíveis, mas também sobre **técnicas de processamento eficazes** que nos ajudem a preparar os dados para o treinamento dos modelos.

In [11]:
# Load the dataset metadata
df = pd.read_csv('E:\\iacd\\3ano\\AC2\\metadata\\UrbanSound8K.csv')

df.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


O dataset fornece principalmente a ``metadata`` sobre as amostras de áudio, incluindo detalhes como o **nome do arquivo** (*slice_file_name*) e os **timestamps de início e fim** (*start* e *end*) no **ID do áudio original** (*fsID*). Além disso, inclui informações sobre o **fold** ao qual cada amostra pertence e o **rótulo** correspondente que a classifica em uma das classes-alvo predefinidas.

Como muito do dataset compreende detalhes técnicos relacionados ao processamento de áudio, nosso foco será analisar a **distribuição dos rótulos alvo no dataset**, uma etapa crucial para entender o **(des)balanceamento de classes** e assegurar que os modelos sejam treinados adequadamente.